In [1]:
import pandas as pd
from add_utils import *
folder = 'C:\\Users\\varun\\OneDrive - Stanford\\Desktop\\Stanford\\3. Spring 25 Quarter\\Independent Research- Geolocalization of Telecom Cables\\mapping_excel_files\\'
mapping1 = pd.read_excel(folder + 'mapping_results_localization_stretch_1_Bridge.xlsx')
mapping2 = pd.read_excel(folder + 'mapping_results_localization_stretch_2.xlsx')
mapping3 = pd.read_excel(folder + 'mapping_results_localization_stretch_3.xlsx')
mapping4 = pd.read_excel(folder + 'mapping_results_localization_stretch_3_clockwise.xlsx')
mapping5= pd.read_excel(folder + 'mapping_results_localization_stretch_3_clockwise_2.xlsx')
# mapping6= pd.read_excel(folder + 'mapping_results_localization_stretch_4_09-22-23.xlsx')
# mapping7= pd.read_excel(folder + 'mapping_results_localization_stretch_5_09-22-23.xlsx')
# mapping8= pd.read_excel(folder + 'mapping_results_localization_stretch_6_09-22-23.xlsx')
# mapping9= pd.read_excel(folder + 'mapping_results_localization_stretch_11_09-22-23.xlsx')

In [2]:
mapping10 = pd.read_excel('mapping_results_localization_stretch_12_09-22-23.xlsx')
mapping9= pd.read_excel('mapping_results_localization_stretch_11_09-22-23.xlsx')
mapping8= pd.read_excel('mapping_results_localization_stretch_6_09-22-23.xlsx')
mapping7= pd.read_excel('mapping_results_localization_stretch_5_09-22-23.xlsx')

In [3]:
# combined_mapping = pd.concat([mapping1, mapping2, mapping3,mapping4,mapping5,mapping6, mapping7, mapping8, mapping9, mapping10], ignore_index=True)
combined_mapping = pd.concat([mapping1, mapping2, mapping3,mapping4,mapping5,mapping7,mapping8,mapping9,mapping10], ignore_index=True)

In [4]:
combined_mapping.to_excel('combined_mapping_results_09-22-23.xlsx', index=False)
combined_mapping.head()  

,chan,Latitude,Longitude,das_time_extrapolated
0,11366,37.272174,-121.828808,2023-09-22 10:06:33.443
1,11367,37.272168,-121.828800,2023-09-22 10:06:33.490
2,11368,37.272162,-121.828792,2023-09-22 10:06:33.535
3,11369,37.272156,-121.828784,2023-09-22 10:06:33.581
4,11370,37.272151,-121.828776,2023-09-22 10:06:33.628


In [5]:
import folium

# Initialize the map centered on the first point
m = folium.Map(
    location=[combined_mapping['Latitude'].iloc[0], combined_mapping['Longitude'].iloc[0]],
    zoom_start=16
)

# Add markers for every 100th point
for _, row in combined_mapping.iloc[::100].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Chan: {row['chan']}<br>Time: {row['das_time_extrapolated']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Display map (or use m.save("filename.html") to save)
m
m.save("combined_mapping_09-22-23.html")


In [85]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(combined_mapping['das_time_extrapolated'], combined_mapping['chan'], marker='.', linestyle='-', markersize=2)
# plt.xlabel('Time')
# plt.ylabel('Channel')
# plt.title('Channel vs Time for Combined Mapping')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [20]:
combined_mapping.head()

,chan,Latitude,Longitude,das_time_extrapolated
0,11366,37.272174,-121.828808,2023-09-22 10:06:33.443
1,11367,37.272168,-121.828800,2023-09-22 10:06:33.490
2,11368,37.272162,-121.828792,2023-09-22 10:06:33.535
3,11369,37.272156,-121.828784,2023-09-22 10:06:33.581
4,11370,37.272151,-121.828776,2023-09-22 10:06:33.628


In [21]:
tap_test_df = pd.read_csv('tap_test_data_cleaned_level_1.csv')
tap_test_df.head()
tap_test_df = tap_test_df.dropna(subset=['Channel'])
tap_test_df.head()
tap_test_df = tap_test_df[['Location', 'Lat', 'Long', 'Channel']]
tap_test_df.head()

,Location,Lat,Long,Channel
0,Outside 1.1,37.336971,-121.886781,254.0
1,3rd/San Fernando,37.335225,-121.886894,669.0
5,1st/Oak,37.322853,-121.880305,2427.0
7,Alma/Monterey,37.316618,-121.873907,3450.0
9,Monterey/Stauffer,37.308410,-121.865710,4820.0


In [7]:
tap_test_df.head()

,Location,Lat,Long,Channel
0,Outside 1.1,37.336971,-121.886781,254.0
1,3rd/San Fernando,37.335225,-121.886894,669.0
5,1st/Oak,37.322853,-121.880305,2427.0
7,Alma/Monterey,37.316618,-121.873907,3450.0
9,Monterey/Stauffer,37.308410,-121.865710,4820.0


In [8]:
# Filter combined_mapping for rows where 'chan' matches any 'Channel' in tap_test_df
filtered_gps = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]
filtered_gps[['chan', 'Latitude', 'Longitude']]
filtered_channels= filtered_gps['chan'].unique()
filtered_channels

array([11240, 12202, 13824, 14752, 17405, 18242, 18740, 19890, 20825,
       19242, 23090, 24640, 26192, 27157, 28757, 30760, 31705],
      dtype=int64)

In [90]:
# Create a DataFrame for filtered channels with GPS coords from both sources and delta columns
filtered_channels_df = pd.DataFrame({
    'Channel': filtered_channels,
    'Latitude_combined': filtered_gps['Latitude'].values,
    'Longitude_combined': filtered_gps['Longitude'].values
})

# Get corresponding GPS from tap_test_df
tap_gps = tap_test_df[tap_test_df['Channel'].isin(filtered_channels)].sort_values('Channel')
filtered_channels_df['Latitude_tap'] = tap_gps['Lat'].values
filtered_channels_df['Longitude_tap'] = tap_gps['Long'].values

# Calculate deltas
filtered_channels_df['del_lat'] = filtered_channels_df['Latitude_combined'] - filtered_channels_df['Latitude_tap']
filtered_channels_df['del_long'] = filtered_channels_df['Longitude_combined'] - filtered_channels_df['Longitude_tap']

filtered_channels_df

,Channel,Latitude_combined,Longitude_combined,Latitude_tap,Longitude_tap,del_lat,del_long
0,12202,37.267532,-121.822232,37.266715,-121.821216,0.000817,-0.001016
1,13824,37.260562,-121.810846,37.260252,-121.810534,0.000310,-0.000312
2,17405,37.251588,-121.817254,37.251631,-121.817754,-0.000043,0.000500
3,18242,37.251526,-121.824831,37.251630,-121.825183,-0.000104,0.000352
4,18740,37.251450,-121.829912,37.251475,-121.830621,-0.000025,0.000709
5,19890,37.258704,-121.830811,37.254654,-121.831019,0.004050,0.000208
6,20825,37.266392,-121.830988,37.258924,-121.830669,0.007468,-0.000319
7,19242,37.254766,-121.830943,37.266608,-121.830932,-0.011842,-0.000011


## ICP
To align `combined_mapping` to `tap_test_df` using ICP (Iterative Closest Point), we treat both as point clouds in latitude/longitude/channel space. ICP will find the best translation and rotation to minimize the distance between corresponding points.

Below is a workflow outline:

ICP Alignment Workflow

1. **Extract Point Clouds**  
    - `tap_test_df`: Use columns `Channel`, `Lat`, `Long` as reference points.
    - `combined_mapping`: Use columns `chan`, `Latitude`, `Longitude` as points to align.

2. **Run ICP Algorithm**  
    - Use an ICP implementation (e.g., from `open3d` or `pycpd`).
    - Find the transformation (translation/rotation) that best aligns `combined_mapping` to `tap_test_df`.

3. **Apply Transformation**  
    - Transform the `combined_mapping` coordinates using the ICP result.

4. **Visualize/Validate**  
    - Plot both datasets before and after alignment to verify the result.

**Note:** ICP works best if both datasets are in the same coordinate system and scale. Latitude/Longitude should be projected to a metric system (e.g., UTM) for accurate alignment.

#### Example Python Code (using Open3D)


This approach aligns the `combined_mapping` to the precise tap test locations. For best results, consider projecting lat/lon to UTM or another metric system before running ICP.

In [91]:
import numpy as np
import open3d as o3d

# Prepare point clouds (pad Z=0 for 3D)
ref_points_3d = np.hstack([tap_test_df[['Lat', 'Long']].dropna().values, np.zeros((tap_test_df.shape[0], 1))])
map_points_3d = np.hstack([combined_mapping[['Latitude', 'Longitude']].values, np.zeros((combined_mapping.shape[0], 1))])

pcd_ref = o3d.geometry.PointCloud()
pcd_ref.points = o3d.utility.Vector3dVector(ref_points_3d)

pcd_map = o3d.geometry.PointCloud()
pcd_map.points = o3d.utility.Vector3dVector(map_points_3d)

threshold = 1e-2
trans_init = np.eye(4)
reg_p2p = o3d.pipelines.registration.registration_icp(
     pcd_map, pcd_ref, threshold, trans_init,
     o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# Apply transformation
pcd_map.transform(reg_p2p.transformation)
transformed_points = np.asarray(pcd_map.points)

# Update combined_mapping with aligned coordinates
combined_mapping['Latitude_aligned'] = transformed_points[:, 0]
combined_mapping['Longitude_aligned'] = transformed_points[:, 1]

print("ICP Transformation Matrix:\n", reg_p2p.transformation)


ICP Transformation Matrix:
 [[ 0.99999848  0.00174517  0.          0.21251872]
 [-0.00174517  0.99999848  0.          0.06524858]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [92]:

combined_mapping.head()

,chan,Latitude,Longitude,das_time_extrapolated,Latitude_aligned,Longitude_aligned
0,11366,37.272174,-121.828808,2023-09-22 10:06:33.443,37.272024,-121.828420
1,11367,37.272168,-121.828800,2023-09-22 10:06:33.490,37.272018,-121.828412
2,11368,37.272162,-121.828792,2023-09-22 10:06:33.535,37.272013,-121.828404
3,11369,37.272156,-121.828784,2023-09-22 10:06:33.581,37.272007,-121.828396
4,11370,37.272151,-121.828776,2023-09-22 10:06:33.628,37.272001,-121.828388


In [93]:
filtered_channels_df['Latitude_aligned'] = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]['Latitude_aligned'].values
filtered_channels_df['Longitude_aligned'] = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]['Longitude_aligned'].values
# Calculate deltas
filtered_channels_df['del_lat_ICP'] = filtered_channels_df['Latitude_aligned'] - filtered_channels_df['Latitude_tap']
filtered_channels_df['del_long_ICP'] = filtered_channels_df['Longitude_aligned'] - filtered_channels_df['Longitude_tap']
filtered_channels_df.head()


,Channel,Latitude_combined,Longitude_combined,Latitude_tap,Longitude_tap,del_lat,del_long,Latitude_aligned,Longitude_aligned,del_lat_ICP,del_long_ICP
0,12202,37.267532,-121.822232,37.266715,-121.821216,0.000817,-0.001016,37.267394,-121.821836,0.000679,-0.000620
1,13824,37.260562,-121.810846,37.260252,-121.810534,0.000310,-0.000312,37.260444,-121.810438,0.000192,0.000096
2,17405,37.251588,-121.817254,37.251631,-121.817754,-0.000043,0.000500,37.251459,-121.816830,-0.000172,0.000924
3,18242,37.251526,-121.824831,37.251630,-121.825183,-0.000104,0.000352,37.251383,-121.824407,-0.000247,0.000776
4,18740,37.251450,-121.829912,37.251475,-121.830621,-0.000025,0.000709,37.251298,-121.829488,-0.000177,0.001133


## Estimating the GPS Time Lag for minimum error from ther Tap Tests - 09/22/2025
- Finding the time lag for which the squared error between the channels 

In [94]:
#Getting the lagged time
filtered_channels_df['lagged_time'] = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]['das_time_extrapolated'].values
t = 0
lag = timedelta(seconds=t)
filtered_channels_df['corrected_time'] = filtered_channels_df['lagged_time'] + lag 
filtered_channels_df.head()
filtered_channels_df['corrected_latitude'] = filtered_channels_df['Latitude_combined']

In [95]:
# Interpolate Latitude and Longitude for corrected_time
corrected_GPS_combined = pd.DataFrame({'corrected_time': filtered_channels_df['corrected_time']})
corrected_GPS_combined.head()


,corrected_time
0,2023-09-22 10:07:15.322
1,2023-09-22 10:08:22.054
2,2023-09-22 10:10:38.586
3,2023-09-22 10:11:16.378
4,2023-09-22 10:11:42.186


In [96]:
corrected_GPS_combined['Channel'] = filtered_channels_df['Channel'].values
corrected_GPS_combined['Latitude'] = np.interp(
    corrected_GPS_combined['corrected_time'].astype(np.int64),
    combined_mapping['das_time_extrapolated'].astype(np.int64),
    combined_mapping['Latitude']
)
corrected_GPS_combined['Longitude'] = np.interp(
    corrected_GPS_combined['corrected_time'].astype(np.int64),
    combined_mapping['das_time_extrapolated'].astype(np.int64),
    combined_mapping['Longitude']
)

corrected_GPS_combined


,corrected_time,Channel,Latitude,Longitude
0,2023-09-22 10:07:15.322,12202,37.267532,-121.822232
1,2023-09-22 10:08:22.054,13824,37.260562,-121.810846
2,2023-09-22 10:10:38.586,17405,37.251588,-121.817254
3,2023-09-22 10:11:16.378,18242,37.251526,-121.824831
4,2023-09-22 10:11:42.186,18740,37.251450,-121.829912
5,2023-09-22 10:12:42.772,19890,37.258704,-121.830811
6,2023-09-22 10:13:31.537,20825,37.266392,-121.830988
7,2023-09-22 10:24:03.841,19242,37.259229,-121.830998


In [97]:
# Match tap_test_df rows to corrected_GPS_combined by Channel
tap_matched = tap_test_df.set_index('Channel').loc[corrected_GPS_combined['Channel']]

# Calculate deltas
corrected_GPS_combined['delta_lat'] = (corrected_GPS_combined['Latitude'] - tap_matched['Lat'].values)**2
corrected_GPS_combined['delta_long'] = (corrected_GPS_combined['Longitude'] - tap_matched['Long'].values)**2

corrected_GPS_combined[['corrected_time', 'Channel', 'Latitude', 'Longitude', 'delta_lat', 'delta_long']]


,corrected_time,Channel,Latitude,Longitude,delta_lat,delta_long
0,2023-09-22 10:07:15.322,12202,37.267532,-121.822232,6.677689e-07,1.033121e-06
1,2023-09-22 10:08:22.054,13824,37.260562,-121.810846,9.621652e-08,9.747872e-08
2,2023-09-22 10:10:38.586,17405,37.251588,-121.817254,1.818251e-09,2.501347e-07
3,2023-09-22 10:11:16.378,18242,37.251526,-121.824831,1.081107e-08,1.240006e-07
4,2023-09-22 10:11:42.186,18740,37.251450,-121.829912,6.392631e-10,5.023581e-07
5,2023-09-22 10:12:42.772,19890,37.258704,-121.830811,4.859011e-08,2.022010e-08
6,2023-09-22 10:13:31.537,20825,37.266392,-121.830988,4.668210e-08,3.164682e-09
7,2023-09-22 10:24:03.841,19242,37.259229,-121.830998,2.092717e-05,4.350687e-10


In [98]:
from datetime import timedelta

# Try a range of lag values (in seconds) to find the minimum sum of squared deltas
best_t = None
min_error = float('inf')

for t_candidate in np.arange(-10, 4,0.001):  # Try lags from -10s to +4s
    lag_candidate = timedelta(seconds=t_candidate)
    corrected_times = filtered_channels_df['lagged_time'] + lag_candidate

    # Interpolate Latitude and Longitude for corrected_times
    lat_interp = np.interp(
        corrected_times.astype(np.int64),
        combined_mapping['das_time_extrapolated'].astype(np.int64),
        combined_mapping['Latitude']
    )
    long_interp = np.interp(
        corrected_times.astype(np.int64),
        combined_mapping['das_time_extrapolated'].astype(np.int64),
        combined_mapping['Longitude']
    )

    # Calculate squared errors
    delta_lat_sq = (lat_interp - filtered_channels_df['Latitude_tap'].values) ** 2
    delta_long_sq = (long_interp - filtered_channels_df['Longitude_tap'].values) ** 2
    total_error = delta_lat_sq.sum() + delta_long_sq.sum()

    if total_error < min_error:
        min_error = total_error
        best_t = t_candidate

print(f"Optimal lag t (seconds): {best_t}")

Optimal lag t (seconds): -4.730000000002921


In [99]:
combined_mapping['das_time_corrected'] = combined_mapping['das_time_extrapolated'] + timedelta(seconds=best_t)
combined_mapping['Latitude_corrected'] = np.interp(
    combined_mapping['das_time_corrected'].astype(np.int64),
    combined_mapping['das_time_extrapolated'].astype(np.int64),
    combined_mapping['Latitude']
)
combined_mapping['Longitude_corrected'] = np.interp(
    combined_mapping['das_time_corrected'].astype(np.int64),
    combined_mapping['das_time_extrapolated'].astype(np.int64),
    combined_mapping['Longitude']
)
combined_mapping.head()

,chan,Latitude,Longitude,das_time_extrapolated,Latitude_aligned,Longitude_aligned,das_time_corrected,Latitude_corrected,Longitude_corrected
0,11366,37.272174,-121.828808,2023-09-22 10:06:33.443,37.272024,-121.828420,2023-09-22 10:06:28.713,37.272174,-121.828808
1,11367,37.272168,-121.828800,2023-09-22 10:06:33.490,37.272018,-121.828412,2023-09-22 10:06:28.760,37.272174,-121.828808
2,11368,37.272162,-121.828792,2023-09-22 10:06:33.535,37.272013,-121.828404,2023-09-22 10:06:28.805,37.272174,-121.828808
3,11369,37.272156,-121.828784,2023-09-22 10:06:33.581,37.272007,-121.828396,2023-09-22 10:06:28.851,37.272174,-121.828808
4,11370,37.272151,-121.828776,2023-09-22 10:06:33.628,37.272001,-121.828388,2023-09-22 10:06:28.898,37.272174,-121.828808


In [100]:
filtered_channels_df['Latitude_corrected_lag'] = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]['Latitude_corrected'].values
filtered_channels_df['Longitude_corrected_lag'] = combined_mapping[combined_mapping['chan'].isin(tap_test_df['Channel'].astype(int))]['Longitude_corrected'].values
# Calculate deltas
filtered_channels_df['del_lat_time_lag_correction'] = filtered_channels_df['Latitude_corrected_lag'] - filtered_channels_df['Latitude_tap']
filtered_channels_df['del_long_time_lag_correction'] = filtered_channels_df['Longitude_corrected_lag'] - filtered_channels_df['Longitude_tap']
filtered_channels_df.head()

,Channel,Latitude_combined,Longitude_combined,Latitude_tap,Longitude_tap,del_lat,del_long,Latitude_aligned,Longitude_aligned,del_lat_ICP,del_long_ICP,lagged_time,corrected_time,corrected_latitude,Latitude_corrected_lag,Longitude_corrected_lag,del_lat_time_lag_correction,del_long_time_lag_correction
0,12202,37.267532,-121.822232,37.266715,-121.821216,0.000817,-0.001016,37.267394,-121.821836,0.000679,-0.000620,2023-09-22 10:07:15.322,2023-09-22 10:07:15.322,37.267532,37.267757,-121.822553,0.001042,-0.001337
1,13824,37.260562,-121.810846,37.260252,-121.810534,0.000310,-0.000312,37.260444,-121.810438,0.000192,0.000096,2023-09-22 10:08:22.054,2023-09-22 10:08:22.054,37.260562,37.261078,-121.811702,0.000826,-0.001168
2,17405,37.251588,-121.817254,37.251631,-121.817754,-0.000043,0.000500,37.251459,-121.816830,-0.000172,0.000924,2023-09-22 10:10:38.586,2023-09-22 10:10:38.586,37.251588,37.251597,-121.816306,-0.000034,0.001448
3,18242,37.251526,-121.824831,37.251630,-121.825183,-0.000104,0.000352,37.251383,-121.824407,-0.000247,0.000776,2023-09-22 10:11:16.378,2023-09-22 10:11:16.378,37.251526,37.251534,-121.823881,-0.000096,0.001302
4,18740,37.251450,-121.829912,37.251475,-121.830621,-0.000025,0.000709,37.251298,-121.829488,-0.000177,0.001133,2023-09-22 10:11:42.186,2023-09-22 10:11:42.186,37.251450,37.251480,-121.829050,0.000005,0.001571


In [104]:
absolute_deltas = deltas_df[['del_lat', 'del_long', 'del_lat_ICP', 'del_long_ICP', 'del_lat_time_lag_correction', 'del_long_time_lag_correction']].abs()
absolute_deltas.sum()

NameError: name 'deltas_df' is not defined

## Spools!

In [ ]:
print(combined_mapping.columns)
print(combined_mapping.loc[:, ["chan", 'Latitude_corrected', 'Longitude_corrected']])

Index(['chan', 'Latitude', 'Longitude', 'das_time_extrapolated',
       'das_time_corrected', 'Latitude_corrected', 'Longitude_corrected'],
      dtype='object')
       chan  Latitude_corrected  Longitude_corrected
0     11366           37.272174          -121.828808
1     11367           37.272174          -121.828808
2     11368           37.272174          -121.828808
3     11369           37.272174          -121.828808
4     11370           37.272174          -121.828808
...     ...                 ...                  ...
8496  19850           37.258100          -121.830579
8497  19851           37.258102          -121.830578
8498  19852           37.258105          -121.830578
8499  19853           37.258108          -121.830578
8500  19854           37.258110          -121.830578

[8501 rows x 3 columns]


In [106]:
Final_mapping = combined_mapping.loc[:, ["chan", 'das_time_corrected', 'Latitude_corrected', 'Longitude_corrected']]
Final_mapping.to_excel('Final_combined_mapping_results_2023-09-22.xlsx', index=False)

In [107]:
# Finding the spools between the channels. These are identified by large jumps in channel numbers with small changes in GPS coordinates.
delta_chan = Final_mapping['chan'].diff().abs()
delta_lat = Final_mapping['Latitude_corrected'].diff().abs()
delta_lon = Final_mapping['Longitude_corrected'].diff().abs()
distance = (delta_lat ** 2 + delta_lon ** 2) ** 0.5
print(distance.shape)
print(delta_chan.shape)
print(Final_mapping.shape)

(8501,)
(8501,)
(8501, 4)


In [108]:
print(distance)

0            NaN
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
8496    0.000003
8497    0.000003
8498    0.000003
8499    0.000003
8500    0.000002
Length: 8501, dtype: float64


In [109]:
# Filter channels where the distance between consecutive corrected GPS points is below a threshold (meters)
# Set threshold in meters
threshold_m = 0.5

# Haversine distance between consecutive points (vectorized)
R = 6371000.0  # Earth radius in meters
lat = np.deg2rad(combined_mapping['Latitude_corrected'].values)
lon = np.deg2rad(combined_mapping['Longitude_corrected'].values)

lat1 = lat[:-1]
lat2 = lat[1:]
lon1 = lon[:-1]
lon2 = lon[1:]

dlat = lat2 - lat1
dlon = lon2 - lon1
a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
distances = 2 * R * np.arcsin(np.sqrt(a))

# Create a Series aligned with combined_mapping (first row has NaN)
dist_series = pd.Series([np.nan] + distances.tolist(), index=combined_mapping.index)
combined_mapping['dist_to_prev_m'] = dist_series

# Filter rows where distance to previous point is less than threshold
filtered_spools_df = combined_mapping[combined_mapping['dist_to_prev_m'] < threshold_m].copy()

# Extract channel list
filtered_spool_channels = filtered_spools_df['chan'].values

# Summary
print(f"Threshold (m): {threshold_m}")
print(f"Filtered points: {len(filtered_spools_df)}")
print("Filtered channels (sample):", filtered_spool_channels[:20])

filtered_spools_df

Threshold (m): 0.5
Filtered points: 1363
Filtered channels (sample): [11367 11368 11369 11370 11371 11372 11373 11374 11375 11376 11377 11378
 11379 11380 11381 11382 11383 11384 11385 11386]


,chan,Latitude,Longitude,das_time_extrapolated,Latitude_aligned,Longitude_aligned,das_time_corrected,Latitude_corrected,Longitude_corrected,dist_to_prev_m
1,11367,37.272168,-121.828800,2023-09-22 10:06:33.490,37.272018,-121.828412,2023-09-22 10:06:28.760,37.272174,-121.828808,0.000000
2,11368,37.272162,-121.828792,2023-09-22 10:06:33.535,37.272013,-121.828404,2023-09-22 10:06:28.805,37.272174,-121.828808,0.000000
3,11369,37.272156,-121.828784,2023-09-22 10:06:33.581,37.272007,-121.828396,2023-09-22 10:06:28.851,37.272174,-121.828808,0.000000
4,11370,37.272151,-121.828776,2023-09-22 10:06:33.628,37.272001,-121.828388,2023-09-22 10:06:28.898,37.272174,-121.828808,0.000000
5,11371,37.272145,-121.828768,2023-09-22 10:06:33.673,37.271996,-121.828380,2023-09-22 10:06:28.943,37.272174,-121.828808,0.000000
...,...,...,...,...,...,...,...,...,...,...
8496,19850,37.259213,-121.830998,2023-09-22 10:23:35.814,37.259060,-121.830587,2023-09-22 10:23:31.084,37.258178,-121.830572,0.299560
8497,19851,37.259217,-121.830998,2023-09-22 10:23:35.790,37.259063,-121.830587,2023-09-22 10:23:31.060,37.258181,-121.830572,0.287579
8498,19852,37.259221,-121.830998,2023-09-22 10:23:35.764,37.259067,-121.830587,2023-09-22 10:23:31.034,37.258184,-121.830571,0.311545
8499,19853,37.259225,-121.830998,2023-09-22 10:23:35.736,37.259072,-121.830588,2023-09-22 10:23:31.006,37.258187,-121.830571,0.335505


In [112]:
# Plot the Combined Mapping
import folium
# Initialize the map centered on the first point
m = folium.Map(
    location=[combined_mapping['Latitude'].iloc[0], combined_mapping['Longitude'].iloc[0]],
    zoom_start=16
)
# Add markers for every 100th point
for _, row in combined_mapping.iloc[::100].iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Chan: {row['chan']}<br>Time: {row['das_time_corrected']}",
        icon=folium.Icon(color='blue', icon='info-sign') 
    ).add_to(m)

m

In [117]:
#Plot the channels and GPS coordinates of the filtered spools using folium
import folium

# Create a folium map centered around the mean coordinates
m = folium.Map(location=[combined_mapping['Latitude'].mean(), combined_mapping['Longitude'].mean()], zoom_start=13)

# Add filtered spool points to the map
for _, row in filtered_spools_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Channel: {row['chan']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# Display the map
m

In [115]:
#Loading the tap test excel file
local_folder = "C:/Users/varun/OneDrive - Stanford/Desktop/Stanford/3. Spring 25 Quarter/Independent Research- Geolocalization of Telecom Cables"
# local_folder = "C:/Users/varun/OneDrive - Stanford/DAS/2023-09-20"
tap_test_excel_path = f"{local_folder}/Tap-test-CCW.xlsx"
tap_test_df = pd.read_excel(tap_test_excel_path)
tap_test_df.columns

Index(['Unnamed: 0', 'Location', 'Lat', 'Long', 'Localized Channels',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

In [ ]:
### Plotting all the GPS coordinates from the Tap Test data
# Convert Long column to float (it's currently string)
tap_test_df['Long'] = pd.to_numeric(tap_test_df['Long'], errors='coerce')

# Drop rows with invalid Lat or Long
tap_test_df = tap_test_df.dropna(subset=['Lat', 'Long'])

# Plot full dataset using function in add_utils.py
plot_tap_test_map(
    df=tap_test_df,
    lat_col='Lat',
    lon_col='Long',
    label_col='Localized Channels',
    map_filename='tap_test_full_map.jpg',
    zoom_start=12
)


Map saved to tap_test_full_map.jpg


In [ ]:
### Plotting all the GPS coordinates from the Tap Test data
# Convert Long column to float (it's currently string)
tap_test_df['Long'] = pd.to_numeric(tap_test_df['Long'], errors='coerce')

# Drop rows with invalid Lat or Long
tap_test_df = tap_test_df.dropna(subset=['Lat', 'Long'])

# Plot full dataset using function in add_utils.py
plot_tap_test_map(
    df=tap_test_df,
    lat_col='Lat',
    lon_col='Long',
    label_col='Localized Channels',
    map_filename='tap_test_full_map.jpg',
    zoom_start=12
)


Map saved to tap_test_full_map.jpg
